In [2]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from anchor_detect import *

# Detect anchor in video list

In [5]:
video_list_path = '../data/face_test.txt'
# test_video_list(video_list_path)
detect_anchor_parallel(None, '../data/anchor_dict_half1.pkl', plot_c=False, nthread=32)

# Collect tmp results

In [3]:
anchor_dict_path = '../data/anchor_dict_half0.pkl'
anchor_dict = {}
anchor_dict_list = []
for i in range(32):
    anchor_dict_list.append('../tmp/anchor_dict_' + str(i) + '.pkl')

for path in anchor_dict_list:
    dict_file = Path(path)
    if not dict_file.is_file():
        continue
    anchor_dict_tmp = pickle.load(open(path, "rb" ))
#         anchor_dict = {**anchor_dict, **anchor_dict_tmp}
    for key, value in anchor_dict_tmp.items():
        anchor_dict[key] = value

pickle.dump(anchor_dict, open(anchor_dict_path, "wb" ), protocol=2)  

# Clean Anchor through multiple shows

In [ ]:
# anchor_dict = pickle.load(open('../data/anchor_dict_test.pkl', 'rb'))
# people_dict = build_people_dict(anchor_dict)
# pickle.dump(people_dict, open('../data/people_dict_half0.pkl', 'wb'), protocol=2)
# people_dict = pickle.load(open('../data/people_dict.pkl', 'rb'))
anchor_dict_clean = clean_anchor_dict(anchor_dict, people_dict)
# pickle.dump(anchor_dict_clean, open('../data/anchor_dict_half0.pkl', 'wb'), protocol=2)

In [16]:
repeat_cnt = []
anchor_dict_new = clean_anchor_dict(anchor_dict, people_dict, repeat_cnt)
pickle.dump(anchor_dict_new, open('../data/anchor_dict.pkl', 'wb'))

In [ ]:
max_cnt = int(np.max(repeat_cnt))    
y = np.bincount(repeat_cnt)
x = np.arange(max_cnt+1)
y = 1. * y / len(repeat_cnt) * 100
plt.figure()
plt.bar(x, y)
plt.xlabel('occurence in other type of shows')
plt.ylabel('percent of videos (%)')
plt.title('candidate anchors shown in other type of shows')

# Stitch all anchor

In [ ]:
import os
images = []
folder = '../tmp/anchor_single/'
for file in sorted(os.listdir(folder)):
    img = cv2.imread(folder + file)
    cv2.rectangle(img, (0, 0), (640, 30), color=(255,255,255), thickness=-1)
    cv2.putText(img, file, (0,25), cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.6, color=(0,0,0), thickness=2)
    if not img is None:
        images.append(img)

images_np = np.zeros((len(images), 480, 640, 3))
for i, im in enumerate(images):
    if im.shape[0] == 480:
        images_np[i] = im
    else:
        images_np[i, :360, :, :] = im
WIDTH = 384
ncol = 5
grid = view_grid(images_np, ncol)
H, W, C = grid.shape
filename = '../tmp/anchor_all.jpg'
grid_small = cv2.resize(grid, (WIDTH*ncol, int(H/W*WIDTH*ncol)))
cv2.imwrite(filename, grid_small)

# Detect anchor in sinlge video

In [ ]:
test_single_video('CNNW_20170502_160000_Inside_Politics', plot_d=True, plot_c=False)

# Check single frame

In [ ]:
video = 'MSNBCW_20170515_150000_MSNBC_Live'
fid = 170
url = 'http://104.198.10.97/frameserver/fetch?path=tvnews%2Fvideos%2F' + video + '.mp4&frame=' + str(fid)
print(url)

# Create host/non-host label

In [7]:
host_dict = {}
for video, anchor_group in anchor_dict.items():
    res = []
    for anchor_person in anchor_group:
        for p in anchor_person:
            res.append((p['fid'], p['bbox']))
    host_dict[video] = res
pickle.dump(host_dict, open('../data/host_dict.pkl', 'wb'), protocol=2)